<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL/blob/main/15_Standings/15_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 15. Team standings
Write a query to return the scores of
each team in the teams table after all
matches displayed in the matches table.
Points are awarded as follows: zero
points for a loss, one point for a tie, and
three points for a win. The result should
include team name and points, and be
ordered by decreasing points.

In [ ]:
import pandas as pd
import numpy  as np

data1 = {'team_id' : [1,2,3,4,5,6],
         'team_name' : ['New York','Atlanta','Chicago','Toronto','Los Angeles','Seattle']
        }

data2 = {'match_id'    : [1,2,3,4,5,6],
         'host_team'   : [1,2,3,4,5,6],
         'guest_team'  : [2,3,4,5,6,1],
         'host_goals'  : [3,2,4,1,2,1],
         'guest_goals' : [0,4,3,1,1,2]
        }

teams   = pd.DataFrame(data1)
matches = pd.DataFrame(data2)

In [ ]:
teams

,team_id,team_name
0,1,New York
1,2,Atlanta
2,3,Chicago
3,4,Toronto
4,5,Los Angeles
5,6,Seattle


In [ ]:
teams.set_index('team_id', inplace = True)

In [ ]:
teams

,team_name
team_id,
1,New York
2,Atlanta
3,Chicago
4,Toronto
5,Los Angeles
6,Seattle


In [ ]:
matches

,match_id,host_team,guest_team,host_goals,guest_goals
0,1,1,2,3,0
1,2,2,3,2,4
2,3,3,4,4,3
3,4,4,5,1,1
4,5,5,6,2,1
5,6,6,1,1,2


In [ ]:
host_matches = matches[['host_team', 'host_goals', 'guest_goals']].copy()
host_matches.rename(columns = {'host_team' : 'team_id'}, inplace = True)
host_matches['host_scores'] = np.where(host_matches['host_goals'] > host_matches['guest_goals'], 3 ,np.where(host_matches['host_goals'] == host_matches['guest_goals'], 1, 0))

In [ ]:
host_matches

,team_id,host_goals,guest_goals,host_scores
0,1,3,0,3
1,2,2,4,0
2,3,4,3,3
3,4,1,1,1
4,5,2,1,3
5,6,1,2,0


In [ ]:
host_scores = pd.DataFrame(host_matches.groupby('team_id')['host_scores'].sum())

In [ ]:
host_scores

,host_scores
team_id,
1,3
2,0
3,3
4,1
5,3
6,0


In [ ]:
guest_matches = matches[['guest_team', 'host_goals', 'guest_goals']].copy()
guest_matches.rename(columns = {'guest_team' : 'team_id'}, inplace = True)
guest_matches['guest_scores'] = np.where(guest_matches['guest_goals'] > guest_matches['host_goals'], 3 ,np.where(guest_matches['guest_goals'] == guest_matches['host_goals'], 1, 0))

In [ ]:
guest_matches

,team_id,host_goals,guest_goals,guest_scores
0,2,3,0,0
1,3,2,4,3
2,4,4,3,0
3,5,1,1,1
4,6,2,1,0
5,1,1,2,3


In [ ]:
guest_scores = pd.DataFrame(guest_matches.groupby('team_id')['guest_scores'].sum())

In [ ]:
guest_scores

,guest_scores
team_id,
1,3
2,0
3,3
4,0
5,1
6,0


In [ ]:
scores = pd.concat([host_scores, guest_scores], axis = 1, join = 'inner')

In [ ]:
scores = pd.DataFrame(scores['host_scores'] + scores['guest_scores'])
scores.rename(columns = {0 : 'score'}, inplace = True)

In [ ]:
scores

,score
team_id,
1,6
2,0
3,6
4,1
5,4
6,0


In [ ]:
results = pd.concat([teams, scores], axis = 1, join = 'inner')

In [ ]:
results.sort_values(by = 'score', ascending = False)

,team_name,score
team_id,,
1,New York,6
3,Chicago,6
5,Los Angeles,4
4,Toronto,1
2,Atlanta,0
6,Seattle,0
